In [66]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
#school_data = pd.read_csv(school_data_to_load)
#student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
#school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

schools_df = pd.read_csv(school_data_to_load, dtype={'School ID' : int, 
                                                'name' : object, 
                                                'type' : object, 
                                                'size' : int, 
                                                'budget' : int})

students_df = pd.read_csv(student_data_to_load, dtype = {'Student ID' : int,
                                                    'name' : object,
                                                    'gender' : object,
                                                    'grade' : object,
                                                    'school' : object,
                                                    'reading_score' : int,
                                                    'math_score' : int})

In [127]:
schools_df.head()
students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


#make sure it's working
schools_df.head(20)
students_df.head(20)

In [67]:
Total_Schools = len(schools_df["School ID"].unique())
Total_Students = len(students_df["Student ID"].unique())
Total_Budget = schools_df["budget"].sum()
Avg_Math_Score = (students_df["math_score"].sum())/len(students_df["math_score"])
Avg_Reading_Score = (students_df["reading_score"].sum())/len(students_df["reading_score"])
Avg_Overall_Score = (students_df["reading_score"].sum()+students_df["math_score"].sum())/((len(students_df["reading_score"]))*2)

passing_stu_math = students_df.loc[students_df['math_score'] >= 70, :]['student_name'].count()
passing_math = (passing_stu_math / Total_Students) * 100

passing_stu_read = students_df.loc[students_df['reading_score'] >= 70, :]['student_name'].count()
passing_read = (passing_stu_read / Total_Students) * 100

District_Summary = pd.DataFrame({
    "Total Schools" : Total_Schools,
    "Total Students" : Total_Students,
    "Total Budget" : Total_Budget,
    "Average Math Score" : Avg_Math_Score,
    "Average Reading Score" : Avg_Reading_Score,
    "Average Overall Score" : Avg_Overall_Score,
    "% Passing Math" : passing_math,
    "% Passing Reading" : passing_read
    
}, index = [0])
#Create a dataframe to hold the above results

District_Summary['Total Budget'] = District_Summary['Total Budget'].map('$ {:,.2f}'.format)
District_Summary['Average Math Score'] = District_Summary['Average Math Score'].map('{:,.2f}%'.format)
District_Summary['Average Reading Score'] = District_Summary['Average Reading Score'].map('{:,.2f}%'.format)
District_Summary['Average Overall Score'] = District_Summary['Average Overall Score'].map('{:,.2f}%'.format)
District_Summary['% Passing Math'] = District_Summary['% Passing Math'].map('{:,.2f}%'.format)
District_Summary['% Passing Reading'] = District_Summary['% Passing Reading'].map('{:,.2f}%'.format)

District_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading
0,15,39170,"$ 24,649,428.00",78.99%,81.88%,80.43%,74.98%,85.81%


In [99]:
total_students_school = students_df.groupby('school_name')['student_name'].count()
total_budget_school = schools_df.groupby('school_name')['budget'].sum()

avg_math_score_school = students_df.groupby('school_name')['math_score'].sum() / total_students_school
avg_reading_score_school = students_df.groupby('school_name')['reading_score'].sum() / total_students_school

passing_math_school = students_df.loc[students_df['math_score'] >= 70, :].groupby('school_name')['math_score'].count()
passing_reading_school = students_df.loc[students_df['reading_score'] >= 70, :].groupby('school_name')['reading_score'].count()

schools_df_addendum = pd.concat([total_students_school, 
                                 total_budget_school, 
                                 avg_math_score_school, 
                                 avg_reading_score_school, 
                                 passing_math_school, 
                                 passing_reading_school], 
                               axis = 1)

schools_df_addendum.reset_index(inplace=True)

schools_df_addendum = schools_df_addendum.rename(columns={'index' : 'school_name',  
                                                        'student_name' : 'Total Students',
                                                        'budget' : 'Total School Budget',
                                                        0 : 'Average Math Score',
                                                        1 : 'Average Reading Score',
                                                        'math_score' : 'passing math raw',
                                                        'reading_score' : 'passing reading raw'
                                                        })

schools_df_addendum = schools_df.merge(schools_df_addendum, on='school_name')

schools_df_addendum['Per Student Budget'] = schools_df_addendum['Total School Budget'] / schools_df_addendum['Total Students']
schools_df_addendum['% Passing Math'] = (schools_df_addendum['passing math raw'] / schools_df_addendum['Total Students']) * 100
schools_df_addendum['% Passing Reading'] = (schools_df_addendum['passing reading raw'] / schools_df_addendum['Total Students']) * 100
schools_df_addendum['Overall Passing Rate'] = (schools_df_addendum['% Passing Math'] + schools_df_addendum['% Passing Reading']) / 2

schools_summary = schools_df_addendum[['school_name', 
                                       'type', 
                                       'Total Students', 
                                       'Total School Budget', 
                                       'Per Student Budget', 
                                       'Average Math Score', 
                                       'Average Reading Score', 
                                       '% Passing Math', 
                                       '% Passing Reading', 
                                       'Overall Passing Rate']]
                                       
schools_summary = schools_summary.rename(columns={'type' : 'School Type'})
schools_summary.set_index('school_name', inplace=True)
del schools_summary.index.name

schools_summary['Total School Budget'] = schools_summary['Total School Budget'].map('$ {:,.2f}'.format)
schools_summary['Per Student Budget'] = schools_summary['Per Student Budget'].map('$ {:,.2f}'.format)
schools_summary['Average Math Score'] = schools_summary['Average Math Score'].map('{:,.2f}%'.format)
schools_summary['Average Reading Score'] = schools_summary['Average Reading Score'].map('{:,.2f}%'.format)
schools_summary['% Passing Math'] = schools_summary['% Passing Math'].map('{:,.2f}%'.format)
schools_summary['% Passing Reading'] = schools_summary['% Passing Reading'].map('{:,.2f}%'.format)
schools_summary['Overall Passing Rate'] = schools_summary['Overall Passing Rate'].map('{:,.2f}%'.format)

schools_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Huang High School,District,2917,"$ 1,910,635.00",$ 655.00,76.63%,81.18%,65.68%,81.32%,73.50%
Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Shelton High School,Charter,1761,"$ 1,056,600.00",$ 600.00,83.36%,83.73%,93.87%,95.85%,94.86%
Hernandez High School,District,4635,"$ 3,022,020.00",$ 652.00,77.29%,80.93%,66.75%,80.86%,73.81%
Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Wilson High School,Charter,2283,"$ 1,319,574.00",$ 578.00,83.27%,83.99%,93.87%,96.54%,95.20%
Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Bailey High School,District,4976,"$ 3,124,928.00",$ 628.00,77.05%,81.03%,66.68%,81.93%,74.31%
Holden High School,Charter,427,"$ 248,087.00",$ 581.00,83.80%,83.81%,92.51%,96.25%,94.38%
Pena High School,Charter,962,"$ 585,858.00",$ 609.00,83.84%,84.04%,94.59%,95.95%,95.27%


In [102]:
#Sort and display the top five schools in overall passing rate
top_performing_schools = schools_summary.sort_values('Overall Passing Rate', ascending=False).iloc[0:5,]
top_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,1858,"$ 1,081,356.00",$ 582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,"$ 1,043,130.00",$ 638.00,83.42%,83.85%,93.27%,97.31%,95.29%
Griffin High School,Charter,1468,"$ 917,500.00",$ 625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$ 585,858.00",$ 609.00,83.84%,84.04%,94.59%,95.95%,95.27%
Wilson High School,Charter,2283,"$ 1,319,574.00",$ 578.00,83.27%,83.99%,93.87%,96.54%,95.20%


In [103]:
#Sort and display the five worst-performing schools
top_performing_schools = schools_summary.sort_values('Overall Passing Rate', ascending=True).iloc[0:5,]
top_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,"$ 2,547,363.00",$ 637.00,76.84%,80.74%,66.37%,80.22%,73.29%
Figueroa High School,District,2949,"$ 1,884,411.00",$ 639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Huang High School,District,2917,"$ 1,910,635.00",$ 655.00,76.63%,81.18%,65.68%,81.32%,73.50%
Johnson High School,District,4761,"$ 3,094,650.00",$ 650.00,77.07%,80.97%,66.06%,81.22%,73.64%
Ford High School,District,2739,"$ 1,763,916.00",$ 644.00,77.10%,80.75%,68.31%,79.30%,73.80%


In [115]:
#Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
#Create a pandas series for each grade. Hint: use a conditional statement.
#Group each series by school
#Combine the series into a dataframe
#Optional: give the displayed data cleaner formatting
#students_df.head()
students_9  = students_df.loc[students_df['grade'] == '9th',]
students_10 = students_df.loc[students_df['grade'] == '10th',]
students_11 = students_df.loc[students_df['grade'] == '11th',]
students_12 = students_df.loc[students_df['grade'] == '12th',]

students_9_school_count  = students_9.groupby('school_name')['grade'].count()
students_10_school_count = students_10.groupby('school_name')['grade'].count()
students_11_school_count = students_11.groupby('school_name')['grade'].count()
students_12_school_count = students_12.groupby('school_name')['grade'].count()

math_9_school  = students_9.groupby('school_name')['math_score'].sum() / students_9_school_count
math_10_school = students_10.groupby('school_name')['math_score'].sum() / students_10_school_count
math_11_school = students_11.groupby('school_name')['math_score'].sum() / students_11_school_count
math_12_school = students_12.groupby('school_name')['math_score'].sum() / students_12_school_count

math_scores_grade_school = pd.concat([math_9_school, 
                                    math_10_school,
                                    math_11_school,
                                    math_12_school],
                                    axis = 1)

math_scores_grade_school = math_scores_grade_school.rename(columns={0 :'9th', 1 : '10th', 2 : '11th', 3 : '12th'})
del math_scores_grade_school.index.name

math_scores_grade_school['9th'] = math_scores_grade_school['9th'].map('{:,.2f}%'.format)
math_scores_grade_school['10th'] = math_scores_grade_school['10th'].map('{:,.2f}%'.format)
math_scores_grade_school['11th'] = math_scores_grade_school['11th'].map('{:,.2f}%'.format)
math_scores_grade_school['12th'] = math_scores_grade_school['12th'].map('{:,.2f}%'.format)

math_scores_grade_school

,9th,10th,11th,12th
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%
Pena High School,83.63%,83.37%,84.33%,84.12%


In [116]:
#same for reading
students_9  = students_df.loc[students_df['grade'] == '9th',]
students_10 = students_df.loc[students_df['grade'] == '10th',]
students_11 = students_df.loc[students_df['grade'] == '11th',]
students_12 = students_df.loc[students_df['grade'] == '12th',]

students_9_school_count  = students_9.groupby('school_name')['grade'].count()
students_10_school_count = students_10.groupby('school_name')['grade'].count()
students_11_school_count = students_11.groupby('school_name')['grade'].count()
students_12_school_count = students_12.groupby('school_name')['grade'].count()

read_9_school  = students_9.groupby('school_name')['reading_score'].sum() / students_9_school_count
read_10_school = students_10.groupby('school_name')['reading_score'].sum() / students_10_school_count
read_11_school = students_11.groupby('school_name')['reading_score'].sum() / students_11_school_count
read_12_school = students_12.groupby('school_name')['reading_score'].sum() / students_12_school_count

read_scores_grade_school = pd.concat([read_9_school, 
                                    read_10_school,
                                    read_11_school,
                                    read_12_school],
                                    axis = 1)

read_scores_grade_school = read_scores_grade_school.rename(columns={0 :'9th', 1 : '10th', 2 : '11th', 3 : '12th'})
del read_scores_grade_school.index.name

read_scores_grade_school['9th'] = read_scores_grade_school['9th'].map('{:,.2f}%'.format)
read_scores_grade_school['10th'] = read_scores_grade_school['10th'].map('{:,.2f}%'.format)
read_scores_grade_school['11th'] = read_scores_grade_school['11th'].map('{:,.2f}%'.format)
read_scores_grade_school['12th'] = read_scores_grade_school['12th'].map('{:,.2f}%'.format)

read_scores_grade_school

,9th,10th,11th,12th
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%
Pena High School,83.81%,83.61%,84.34%,84.59%


In [140]:
#accidently skipped spending one, that one is next

# Sample bins. Feel free to create your own bins.

size_df = schools_df
size_students_df = students_df

size_df['Per Student Budget'] = size_df['budget'] / size_df['size']

bins = [0, 1000, 2000, 5000]
group_labels = ['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']
size_df['School Sizes'] = pd.cut(size_df['size'],bins,labels=group_labels)

size_students_df = size_students_df.merge(size_df, left_on='school_name', right_on='school_name')

size_students_df.drop(['student_name'], axis=1, inplace=True)

students_by_size = size_students_df.groupby('School Sizes')['school_name'].count()

avg_math_score_size = size_students_df.groupby('School Sizes')['math_score'].sum() / students_by_size
avg_reading_score_size = size_students_df.groupby('School Sizes')['reading_score'].sum() / students_by_size

passing_math_size = (size_students_df.loc[size_students_df['math_score'] >= 70, :].groupby('School Sizes')['math_score'].count() / students_by_size) * 100
passing_reading_size = (size_students_df.loc[size_students_df['reading_score'] >= 70, :].groupby('School Sizes')['reading_score'].count() / students_by_size) * 100


scores_by_size = pd.concat([avg_math_score_size, 
                                 avg_reading_score_size, 
                                 passing_math_size, 
                                 passing_reading_size], 
                               axis = 1)

scores_by_size = scores_by_size.rename(columns={0 : 'Average Math Score',
                                                       1 : 'Average Reading Score',
                                                       2 : '% Passing Math',
                                                       3 : '% Passing Reading'})

scores_by_size['Overall Passing Rate'] = (scores_by_size['% Passing Math'] + scores_by_size['% Passing Reading']) / 2

scores_by_size.index.rename('School Sizes', inplace=True)

scores_by_size['Average Math Score'] = scores_by_size['Average Math Score'].map('{:,.2f}%'.format)
scores_by_size['Average Reading Score'] = scores_by_size['Average Reading Score'].map('{:,.2f}%'.format)
scores_by_size['% Passing Math'] = scores_by_size['% Passing Math'].map('{:,.2f}%'.format)
scores_by_size['% Passing Reading'] = scores_by_size['% Passing Reading'].map('{:,.2f}%'.format)
scores_by_size['Overall Passing Rate'] = scores_by_size['Overall Passing Rate'].map('{:,.2f}%'.format)

scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Sizes,,,,,
Small (<1000),83.83%,83.97%,93.95%,96.04%,95.00%
Medium (1000-2000),83.37%,83.87%,93.62%,96.77%,95.19%
Large (2000-5000),77.48%,81.20%,68.65%,82.13%,75.39%


In [141]:
spending_df = schools_df
spending_students_df = students_df

spending_df['Per Student Budget'] = spending_df['budget'] / spending_df['size']

bins = [0, 585, 615, 645, 675]
group_labels = ['<$585','$585-615','$615-645','$645-675']
spending_df['Spending Ranges Per Student'] = pd.cut(spending_df['Per Student Budget'],bins,labels=group_labels)

spending_students_df = spending_students_df.merge(spending_df, left_on='school_name', right_on='school_name')

students_by_budget = spending_students_df.groupby('Spending Ranges Per Student')['student_name'].count()

avg_math_score_budget = spending_students_df.groupby('Spending Ranges Per Student')['math_score'].sum() / students_by_budget
avg_reading_score_budget = spending_students_df.groupby('Spending Ranges Per Student')['reading_score'].sum() / students_by_budget

passing_math_budget = (spending_students_df.loc[spending_students_df['math_score'] > 60, :].groupby('Spending Ranges Per Student')['math_score'].count() / students_by_budget) * 100
passing_reading_budget = (spending_students_df.loc[spending_students_df['reading_score'] > 60, :].groupby('Spending Ranges Per Student')['reading_score'].count() / students_by_budget) * 100

scores_by_spending = pd.concat([avg_math_score_budget, 
                                 avg_reading_score_budget, 
                                 passing_math_budget, 
                                 passing_reading_budget], 
                               axis = 1)

scores_by_spending = scores_by_spending.rename(columns={0 : 'Average Math Score',
                                                       1 : 'Average Reading Score',
                                                       2 : '% Passing Math',
                                                       3 : '% Passing Reading'})

scores_by_spending['Overall Passing Rate'] = (scores_by_spending['% Passing Math'] + scores_by_spending['% Passing Reading']) / 2

scores_by_spending.index.rename('Spending Ranges Per Student', inplace=True)

scores_by_spending['Average Math Score'] = scores_by_spending['Average Math Score'].map('{:,.2f}%'.format)
scores_by_spending['Average Reading Score'] = scores_by_spending['Average Reading Score'].map('{:,.2f}%'.format)
scores_by_spending['% Passing Math'] = scores_by_spending['% Passing Math'].map('{:,.2f}%'.format)
scores_by_spending['% Passing Reading'] = scores_by_spending['% Passing Reading'].map('{:,.2f}%'.format)
scores_by_spending['Overall Passing Rate'] = scores_by_spending['Overall Passing Rate'].map('{:,.2f}%'.format)

scores_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges Per Student,,,,,
<$585,83.36%,83.96%,100.00%,100.00%,100.00%
$585-615,83.53%,83.84%,100.00%,100.00%,100.00%
$615-645,78.06%,81.43%,89.21%,100.00%,94.60%
$645-675,77.05%,81.01%,86.64%,100.00%,93.32%


In [144]:
type_students_df = size_students_df

students_by_type = type_students_df.groupby('type')['school_name'].count()

avg_math_score_type = type_students_df.groupby('type')['math_score'].sum() / students_by_type
avg_reading_score_type = type_students_df.groupby('type')['reading_score'].sum() / students_by_type

passing_math_type = (type_students_df.loc[type_students_df['math_score'] > 60, :].groupby('type')['math_score'].count() / students_by_type) * 100
passing_reading_type = (type_students_df.loc[type_students_df['reading_score'] > 60, :].groupby('type')['reading_score'].count() / students_by_type) * 100


scores_by_type = pd.concat([avg_math_score_type, 
                                 avg_reading_score_type, 
                                 passing_math_type, 
                                 passing_reading_type], 
                               axis = 1)

scores_by_type = scores_by_type.rename(columns={0 : 'Average Math Score',
                                                       1 : 'Average Reading Score',
                                                       2 : '% Passing Math',
                                                       3 : '% Passing Reading'})

scores_by_type['Overall Passing Rate'] = (scores_by_type['% Passing Math'] + scores_by_type['% Passing Reading']) / 2

scores_by_type.index.rename('School Type', inplace=True)

scores_by_type['Average Math Score'] = scores_by_type['Average Math Score'].map('{:,.2f}%'.format)
scores_by_type['Average Reading Score'] = scores_by_type['Average Reading Score'].map('{:,.2f}%'.format)
scores_by_type['% Passing Math'] = scores_by_type['% Passing Math'].map('{:,.2f}%'.format)
scores_by_type['% Passing Reading'] = scores_by_type['% Passing Reading'].map('{:,.2f}%'.format)
scores_by_type['Overall Passing Rate'] = scores_by_type['Overall Passing Rate'].map('{:,.2f}%'.format)

scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.41%,83.90%,100.00%,100.00%,100.00%
District,76.99%,80.96%,86.80%,100.00%,93.40%
